In [1]:
import pandas as pd
import tensorflow as tf
import numpy

In [2]:
model = tf.keras.models.load_model('modelos/NNModelGS2')

In [3]:
df = pd.read_csv('test_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

In [4]:
df = df.drop(columns=["geo_level_2_id", "geo_level_3_id"])

In [5]:
def one_hot_encode_data(dataframe, column_name):
    dummies = pd.get_dummies(dataframe[column_name])
    rename_columns = {}
    for column in dummies.columns.values:
        rename_columns[column] = column_name + '_' + column
    return dataframe.drop(columns=[column_name]).join(dummies.rename(columns=rename_columns))

In [6]:
df = one_hot_encode_data(df,'land_surface_condition')
df = one_hot_encode_data(df,'foundation_type')
df = one_hot_encode_data(df,'roof_type')
df = one_hot_encode_data(df,'ground_floor_type')
df = one_hot_encode_data(df,'other_floor_type')
df = one_hot_encode_data(df,'position')
df = one_hot_encode_data(df,'plan_configuration')
df = one_hot_encode_data(df,'legal_ownership_status')
df = one_hot_encode_data(df,'geo_level_1_id')
df

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,geo_level_1_id_28,geo_level_1_id_29,geo_level_1_id_3,geo_level_1_id_30,geo_level_1_id_4,geo_level_1_id_5,geo_level_1_id_6,geo_level_1_id_7,geo_level_1_id_8,geo_level_1_id_9
building_id,,,,,,,,,,,,,,,,,,,,,
300051,3,20,7,6,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99355,2,25,13,5,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
890251,2,5,4,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
745817,1,0,19,3,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
421793,3,15,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,3,70,20,6,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
663567,3,25,6,7,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1049160,1,50,3,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
test_database = df.to_numpy(dtype=numpy.int64)
predictions = model(test_database).numpy()
building_ids = df.reset_index()['building_id']
results = []
for line in predictions:
    if line[0] > line [1] and line [0] > line [2]:
        results.append(1)
    elif line[1] > line[2]:
        results.append(2)
    else:
        results.append(3)
result_series = pd.Series(results).rename("damage_grade")
my_submission = pd.merge(building_ids,result_series,left_index=True,right_index=True).set_index('building_id')

In [8]:
my_submission.to_csv('submission_feda_3.csv')